In [3]:
import os
# Local Imports
from src.seg2map import map_interface
from src.seg2map import log_maker #must be the first module loaded to create logs folder

# External Imports
import ee
from google.auth import exceptions as google_auth_exceptions

# suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Authenticate and Initialize with Google Earth Engine (GEE)

- Run this cell to initialize with GEE which will allow you to download remote sensing data from GEE.

### First Time Users

- In order to use Google Earth Engine (GEE) you will need to sign up to request access to use Google Earth Engine.https://signup.earthengine.google.com. You will only need to do this once and it takes only a day to get your account verified.

### How `ee.Authenticate()` works

- In order to initialize with GEE you will need an authorization token with is obtained by running `ee.Authenticate()`.This token lasts 7 days and during those 7 days you will not need to authenticate with google earth engine with an access code. Once the 7 days are up you will need to reauthenticate to use GEE again.


In [4]:
try:
    ee.Initialize()
except google_auth_exceptions.RefreshError as exception:
    print("Please authenticate with Google:\n")
    ee.Authenticate()
    ee.Initialize()

# How to Use The Map

---
1.  Click `Save Settings` Button
2. Use the rectangle tool to draw a ROI along the coastline.
3. Load transects into your bounding box with the `Load Transects` button. If any exist for the bounding box you selected they should appear.
4. Click the ROIs you want to download.
5. Once you've selected all the ROIs you want to download click `Downlod Imagery`
 - If any of the ROIs succesfully download they will have their own folder with all their data in the `data` directory in the `seg2map` directory

In [5]:
# from coastseg.map_UI import UI
from src.seg2map.map_UI import UI
from src.seg2map.map_interface import Seg2Map

seg2map=Seg2Map()
seg2map_ui = UI(seg2map)
seg2map_ui.create_dashboard()

HBox()

HBox()

In [1]:
import sys
print(sys.executable)

C:\Users\Sharon\anaconda3\envs\seg2map\python.exe


In [2]:
import sys
print(sys.version)

3.10.9 | packaged by conda-forge | (main, Jan 11 2023, 15:15:40) [MSC v.1916 64 bit (AMD64)]


In [3]:
from osgeo import gdal

In [ ]:
print(list(map(lambda x:x.name, seg2map.map.layers)))

In [ ]:
remove_layers = [layer for layer in seg2map.original_layers + seg2map.seg_layers if layer in seg2map.map.layers]
print(list(map(lambda x:x.name,remove_layers)))

In [ ]:
remove_layers = [layer for layer in seg2map.original_layers + seg2map.seg_layers if layer in seg2map.map.layers]
print(list(map(lambda x:x.name,remove_layers)))

In [ ]:
layers = [layer for layer in seg2map.original_layers + seg2map.seg_layers]
print(list(map(lambda x:x.name,layers)))

In [ ]:
from src.seg2map import log_maker
from src.seg2map.models_UI import UI_Models
models_ui = UI_Models()
models_ui.create_dashboard()

In [ ]:
from PIL import Image, ImageSequence
img_path = r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\share\multiband0_USDA_NAIP_DOQQ_m_4012415_ne_10_1_20100629.png'

In [ ]:
from PIL import Image
import numpy as np
import os
im = Image.open(img_path, formats=("PNG",))
array3 = np.array(im)
reshaped_array3 =array3.reshape(-1, array3.shape[2])
unqiue_colors = np.unique(reshaped_array3,axis=0)
print(unqiue_colors)

In [ ]:
from PIL import Image, ImageSequence
img_path = r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\share\multiband0_USDA_NAIP_DOQQ_m_4012415_ne_10_1_20100629.png'
im = Image.open(img_path, formats=("PNG",)).convert("RGB")
out_path = img_path.replace(".png", "_RGB.png")

In [ ]:
import numpy as np
array3 = np.array(im)

In [ ]:
reshaped_array3 =array3.reshape(-1, array3.shape[2])
unqiue_colors = np.unique(reshaped_array3,axis=0)
print(unqiue_colors)

In [ ]:
import os
os.path.exists(out_path)

In [ ]:
from osgeo import gdal
gdal.UseExceptions() 

In [ ]:
import os
from osgeo import gdal
from typing import List


def build_vrt(vrt: str, files: List[str], resample_name: str) -> None:
    """builds .vrt file which will hold information needed for overlay
    Args:
        vrt (:obj:`string`): name of vrt file, which will be created
        files (:obj:`list`): list of file names for merging
        resample_name (:obj:`string`): name of resampling method
    """

    options = gdal.BuildVRTOptions(srcNodata=0)
    virtual_dataset=gdal.BuildVRT(destName=vrt, srcDSOrSrcDSTab=files, options=options)
    virtual_dataset.FlushCache()
    virtual_dataset = None
    print(os.path.exists(vrt))
    add_pixel_fn(vrt, resample_name)



def add_pixel_fn(filename: str, resample_name: str) -> None:
    """inserts pixel-function into vrt file named 'filename'
    Args:
        filename (:obj:`string`): name of file, into which the function will be inserted
        resample_name (:obj:`string`): name of resampling method
    """

    header = """  <VRTRasterBand dataType="Byte" band="1" subClass="VRTDerivedRasterBand">"""
    contents = """
    <PixelFunctionType>{0}</PixelFunctionType>
    <PixelFunctionLanguage>Python</PixelFunctionLanguage>
    <PixelFunctionCode><![CDATA[{1}]]>
    </PixelFunctionCode>"""

    lines = open(filename, 'r').readlines()
    lines[3] = header  # FIX ME: 3 is a hand constant
    lines.insert(4, contents.format(resample_name,
                                    get_resample(resample_name)))
    open(filename, 'w').write("".join(lines))


def get_resample(name: str) -> str:
    """retrieves code for resampling method
    Args:
        name (:obj:`string`): name of resampling method
    Returns:
        method :obj:`string`: code of resample method
    """

    methods = {
        "first":
        """
import numpy as np
def first(in_ar, out_ar, xoff, yoff, xsize, ysize, raster_xsize,raster_ysize, buf_radius, gt, **kwargs):
    y = np.ones(in_ar[0].shape)
    for i in reversed(range(len(in_ar))):
        mask = in_ar[i] == 0
        y *= mask
        y += in_ar[i]
    np.clip(y,0,255, out=out_ar)
""",
        "last":
        """
import numpy as np
def last(in_ar, out_ar, xoff, yoff, xsize, ysize, raster_xsize,raster_ysize, buf_radius, gt, **kwargs):
    y = np.ones(in_ar[0].shape)
    for i in range(len(in_ar)):
        mask = in_ar[i] == 0
        y *= mask
        y += in_ar[i]
    np.clip(y,0,255, out=out_ar)
""",
        "max":
        """
import numpy as np
def max(in_ar, out_ar, xoff, yoff, xsize, ysize, raster_xsize,raster_ysize, buf_radius, gt, **kwargs):
    y = np.max(in_ar, axis=0)
    np.clip(y,0,255, out=out_ar)
""",
        "average":
        """
import numpy as np
def average(in_ar, out_ar, xoff, yoff, xsize, ysize, raster_xsize,raster_ysize, buf_radius, gt, **kwargs):
    div = np.zeros(in_ar[0].shape)
    for i in range(len(in_ar)):
        div += (in_ar[i] != 0)
    div[div == 0] = 1
    
    y = np.sum(in_ar, axis = 0, dtype = 'uint16')
    y = y / div
    
    np.clip(y,0,255, out = out_ar)
"""}

    if name not in methods:
        raise ValueError(
            "ERROR: Unrecognized resampling method (see documentation): '{}'.".
            format(name))

    return methods[name]

In [ ]:
def merge_files(src_files: str, dest_path: str, create_jpg: bool = True) -> str:
    """Merge a list of GeoTIFF files into a single JPEG file.

    Args:
    src_files (List[str]): A list of file paths to be merged.
    dest_path (str): The path to the output JPEG file.

    Returns:
    str: The path to the output JPEG file.
    """
    # Check if path to source exists
    for file in src_files:
        if not os.path.exists(file):
            raise FileNotFoundError(f"{file} not found.")
    try:
        ## create vrt(virtual world format) file
        # Create VRT file
        vrt_options = gdal.BuildVRTOptions(
            resampleAlg="mode", srcNodata=0, VRTNodata=0
        )

        virtual_dataset = gdal.BuildVRT(dest_path, src_files, options=vrt_options)
        # flushing the cache causes the vrt file to be created
        virtual_dataset.FlushCache()
        # reset the dataset object
        virtual_dataset = None
        # print(f"dest_path: {dest_path}")
        # parent_dir = os.path.dirname(dest_path)
        # tmp_path = os.path.join(parent_dir,'tmp.vrt')
        # print(f"tmp_path: {tmp_path}")
        
        # build_vrt(tmp_path, src_files, 'max')
        
        # gdal.SetConfigOption('GDAL_VRT_ENABLE_PYTHON', 'YES')

        # print(f"dest_path: {dest_path}")
        # kwargs = {
        #     'format': 'GTiff',
        #     'outputType': gdal.GDT_Byte
        # }
        # virtual_dataset = gdal.Translate(destName=dest_path, srcDS=tmp_path,kwargs)
        # virtual_dataset.FlushCache()
        # virtual_dataset = None
        # print(os.path.exists(dest_path))


        # print(f"after new  build_vrt dest_path: {dest_path}")

        # create geotiff (.tiff) from merged vrt file
        tif_path = dest_path.replace(".vrt", ".tif")
        print(f"tif_path: {tif_path}")
        virtual_dataset = gdal.Translate(
            tif_path,
            creationOptions=["COMPRESS=LZW", "TILED=YES"],
            srcDS=dest_path,
        )
        virtual_dataset.FlushCache()
        virtual_dataset = None
        print(f"dest_path(tif_path): {tif_path}")

        if create_jpg:
            # convert .vrt to .jpg file
            virtual_dataset = gdal.Translate(
                dest_path.replace(".vrt", ".jpg"),
                creationOptions=["WORLDFILE=YES", "QUALITY=100"],
                srcDS=dest_path.replace(".vrt", ".tif"),
            )
            virtual_dataset.FlushCache()
            virtual_dataset = None
            print(f"dest_path: {dest_path.replace('.vrt', '.jpg')}")

        return dest_path
    except Exception as e:
        print(e)
        raise e

In [ ]:
def merge_files(src_files: str, dest_path: str, create_jpg: bool = True) -> str:
    """Merge a list of GeoTIFF files into a single JPEG file.

    Args:
    src_files (List[str]): A list of file paths to be merged.
    dest_path (str): The path to the output JPEG file.

    Returns:
    str: The path to the output JPEG file.
    """
    # Check if path to source exists
    for file in src_files:
        if not os.path.exists(file):
            raise FileNotFoundError(f"{file} not found.")
    try:
        ## create vrt(virtual world format) file
        # Create VRT file
        vrt_options = gdal.BuildVRTOptions(
            resampleAlg="nearest", srcNodata=0, VRTNodata=0
        )

        virtual_dataset = gdal.BuildVRT(dest_path, src_files, options=vrt_options)
        # flushing the cache causes the vrt file to be created
        virtual_dataset.FlushCache()
        # reset the dataset object
        virtual_dataset = None
        # create geotiff (.tiff) from merged vrt file
        tif_path = dest_path.replace(".vrt", ".tif")
        print(f"tif_path: {tif_path}")
        virtual_dataset = gdal.Translate(
            tif_path,
            creationOptions=["COMPRESS=LZW", "TILED=YES"],
            srcDS=dest_path,
        )
        virtual_dataset.FlushCache()
        virtual_dataset = None
        print(f"dest_path(tif_path): {tif_path}")

        if create_jpg:
            # convert .vrt to .jpg file
            virtual_dataset = gdal.Translate(
                dest_path.replace(".vrt", ".jpg"),
                creationOptions=["WORLDFILE=YES", "QUALITY=100"],
                srcDS=dest_path.replace(".vrt", ".tif"),
            )
            virtual_dataset.FlushCache()
            virtual_dataset = None
            print(f"dest_path: {dest_path.replace('.vrt', '.jpg')}")

        return dest_path
    except Exception as e:
        print(e)
        raise e

In [ ]:
from glob import glob
import os
from osgeo import gdal

src_path=r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010'
tif_files = glob(os.path.join(src_path, "*.tif"))
tif_files = [file for file in tif_files if "merged_multispectral" not in file]

tif_files[0]

In [ ]:
dst_path = os.path.join(src_path, "merged_multispectral_fruit_4.vrt")
merged_file = merge_files([tif_files[0],tif_files[0]], dst_path, create_jpg=True)

In [ ]:
from PIL import Image
import numpy as np

# Example usage
input_filename =r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010\merged_multispectral_fruit_4.jpg'

# Load the input image
with Image.open(input_filename) as image:
    # Convert the image to a NumPy array
    array3 = np.array(image)
reshaped_array3 =array3.reshape(-1, array3.shape[2])
unique_colors = np.unique(reshaped_array3,axis=0)
print(len(unique_colors))

In [4]:
# Mosaic.jpg
from PIL import Image
import numpy as np

# Example usage
input_filename =r'C:\1_USGS\4_seg2map\seg2map\image_mosaic-20230306T164950Z-001\image_mosaic\MosaicOEM.jpg'

# Load the input image
with Image.open(input_filename) as image:
    # Convert the image to a NumPy array
    array3 = np.array(image)
reshaped_array3 =array3.reshape(-1, array3.shape[2])
unique_colors = np.unique(reshaped_array3,axis=0)
print(len(unique_colors))

211343


In [3]:
from osgeo import gdal

kwargs = {
        'format': 'JPEG',
        'outputType': gdal.GDT_Byte
}

def gdal_translate_jpeg(f, kwargs):
    ds = gdal.Translate(f.replace('.tif','.jpg'), f, **kwargs)
    ds = None # close and save ds

input_filename =r'C:\1_USGS\4_seg2map\seg2map\black_and_white.tif'
gdal_translate_jpeg(input_filename,kwargs)


In [20]:
# Mosaic.jpg
from PIL import Image
import numpy as np

file =r'C:\1_USGS\4_seg2map\seg2map\data\fresh_downloads\ID_NVBbrh_dates_2010-01-01_to_2013-12-31\multiband\2010\Mosaic.jpg'

# Load the input image
with Image.open(file) as image:
    # Convert the image to a NumPy array
    array3 = np.array(image)
array3.shape
# reshaped_array3 =array3.reshape(-1, array3.shape[1])
# unique_colors = np.unique(reshaped_array3,axis=0)
# print(len(unique_colors))

(1764, 3059)

In [21]:
np.unique(array3)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [18]:
# Mosaic.jpg
from PIL import Image
import numpy as np

file =r'C:\1_USGS\4_seg2map\seg2map\data\fresh_downloads_script_ran1\ID_NVBbrh_dates_2010-01-01_to_2013-12-31\multiband\2010\Mosaic.tif'

# Load the input image
with Image.open(file) as image:
    # Convert the image to a NumPy array
    array3 = np.array(image)
array3.shape
# reshaped_array3 =array3.reshape(-1, array3.shape[1])
# unique_colors = np.unique(reshaped_array3,axis=0)
# print(len(unique_colors))

(1764, 3059)

In [19]:
np.unique(array3)

array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [11]:
reshaped_array3 =array3.reshape(-1, array3.shape[0])
reshaped_array3

array([[3, 3, 3, ..., 3, 3, 3],
       [3, 3, 4, ..., 7, 7, 7],
       [7, 7, 7, ..., 2, 2, 2],
       ...,
       [7, 7, 7, ..., 6, 6, 6],
       [6, 6, 3, ..., 4, 4, 4],
       [4, 4, 4, ..., 3, 3, 3]], dtype=uint8)

In [12]:
reshaped_array3.shape

(3059, 1764)

In [10]:
# Mosaic.jpg
from PIL import Image
import numpy as np

# Example usage
# input_filename =r'C:\1_USGS\4_seg2map\seg2map\image_mosaic-20230306T164950Z-001\image_mosaic\MosaicXBD.jpg'
input_filename =r'C:\1_USGS\4_seg2map\seg2map\image_mosaic-20230306T164950Z-001\image_mosaic\MosaicCHESA.tif'
# input_filename = r'C:\1_USGS\4_seg2map\seg2map\data\fresh_downloads\ID_NVBbrh_dates_2010-01-01_to_2013-12-31\multiband\2010\merged_multispectral.tif'

# Load the input image
with Image.open(input_filename) as image:
    # Convert the image to a NumPy array
    array3 = np.array(image)
reshaped_array3 =array3.reshape(-1, array3.shape[2])
unique_colors = np.unique(reshaped_array3,axis=0)
print(len(unique_colors))

17


In [6]:
unique_colors

array([[  0,   0,   0],
       [  0,   0,   2],
       [  0,   0,   4],
       ...,
       [252,  47,   0],
       [252,  48,   0],
       [254,  47,   0]], dtype=uint8)

In [7]:
reshaped_array3

array([[222,  56,  16],
       [220,  56,  18],
       [217,  58,  19],
       ...,
       [  0,   0,   0],
       [  0,   0,   0],
       [  0,   0,   0]], dtype=uint8)

In [ ]:
# with mode
unique_colors

In [ ]:
# with nearest
unique_colors

In [ ]:
# Get imgs
imgsToMosaic=[]
sorted_imgs = sorted(glob(os.path.join(inDir, '*.png')))
for img in sorted_imgs:
    if 'overlay' not in img:
        print(img)
        imgsToMosaic.append(img)
imgsToMosaic

In [ ]:
import sys, os
from osgeo import gdal
from glob import glob

# Params
inDir = r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\share'
outVRT = os.path.join(inDir, 'Mosaic.vrt')
outTIF = os.path.join(inDir, 'Mosaic.tif')
outJPG = os.path.join(inDir, 'Mosaic.jpg')

# Get imgs
imgsToMosaic=[]
sorted_imgs = sorted(glob(os.path.join(inDir, '*.png')))
for img in sorted_imgs:
    if 'overlay' not in img:
        imgsToMosaic.append(img)
        
print(imgsToMosaic)
# First build vrt
vrt_options = gdal.BuildVRTOptions(resampleAlg='mode')
gdal.BuildVRT(outVRT, imgsToMosaic, options=vrt_options)

# Create GeoTiff from vrt
ds = gdal.Open(outVRT)

kwargs = {'format': 'GTiff', 'creationOptions': ['NUM_THREADS=ALL_CPUS', 'COMPRESS=LZW']}
gdal.Translate(outTIF, ds, **kwargs)

# Create jpg from vrt
kwargs = {'format': 'GTiff', 'creationOptions': ['NUM_THREADS=ALL_CPUS', 'COMPRESS=LZW']}
gdal.Translate(outJPG, ds, **kwargs)

In [ ]:
unique_colors

In [ ]:
reshaped_array3.shape

In [ ]:
[51, 102, 204] in 

In [ ]:
colors = np.array([[0, 153, 198],
                [16, 150, 24],
                [51, 102, 204],
                [102, 170, 0],
                [153, 0, 153],
                [184, 46, 46],
                [220, 57, 18],
                [221, 68, 119],
                [255, 153, 0]])
for color in colors:
    print(color)

In [ ]:
colors = np.array([[0, 153, 198],
                [16, 150, 24],
                [51, 102, 204],
                [102, 170, 0],
                [153, 0, 153],
                [184, 46, 46],
                [220, 57, 18],
                [221, 68, 119],
                [255, 153, 0]])
for color in colors:

    target = np.array(color)

    # Check if target exists in the array
    mask = np.all(unique_colors  == target, axis=1)

    if np.any(mask):
        # Get the index of the first matching element
        index = np.where(mask)[0][0]
        print(f"Target:{target} found at index {index}.")
    else:
        print(f"Target:{target} not found in the array.")

In [ ]:
target = np.array([  0, 153, 198])

# [[  0 153 198]
#  [ 16 150  24]
#  [ 51 102 204]
#  [102 170   0]
#  [153   0 153]
#  [184  46  46]
#  [220  57  18]
#  [221  68 119]
#  [255 153   0]]

# Check if target exists in the array
mask = np.all(unique_colors  == target, axis=1)

if np.any(mask):
    # Get the index of the first matching element
    index = np.where(mask)[0][0]
    print(f"Target found at index {index}.")
else:
    print("Target not found in the array.")

In [ ]:
unique_colors

In [ ]:
unique_colors[3262]

In [ ]:
unique_colors

In [ ]:
from glob import glob
import os
from osgeo import gdal

src_path=r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010'
tif_files = glob(os.path.join(src_path, "*.tif"))
tif_files = [file for file in tif_files if "merged_multispectral" not in file]

tif_files[2:]

In [ ]:
dst_path = os.path.join(src_path, "merged_multispectral_fruit_3.vrt")
merged_file = merge_files(tif_files[2:], dst_path, create_jpg=True)

In [ ]:
from PIL import Image
import numpy as np

# Example usage
input_filename =r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010\merged_multispectral_fruit_3.jpg'

# Load the input image
with Image.open(input_filename) as image:
    # Convert the image to a NumPy array
    array3 = np.array(image)
reshaped_array3 =array3.reshape(-1, array3.shape[2])
unique_colors = np.unique(reshaped_array3,axis=0)
print(len(unique_colors))

In [ ]:
from glob import glob
import os
from osgeo import gdal

src_path=r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010'
tif_files = glob(os.path.join(src_path, "*.tif"))
tif_files = [file for file in tif_files if "merged_multispectral" not in file]

tif_files[:2]

In [ ]:
dst_path = os.path.join(src_path, "merged_multispectral_fruit_2.vrt")
merged_file = merge_files(tif_files[:2], dst_path, create_jpg=True)

In [ ]:
from PIL import Image
import numpy as np

# Example usage
input_filename =r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010\merged_multispectral_fruit_2.jpg'

# Load the input image
with Image.open(input_filename) as image:
    # Convert the image to a NumPy array
    array3 = np.array(image)
reshaped_array3 =array3.reshape(-1, array3.shape[2])
unique_colors = np.unique(reshaped_array3,axis=0)
print(len(unique_colors))

In [ ]:
print(len(unique_colors))

In [ ]:
def merge_files(src_files: str, dest_path: str, create_jpg: bool = True) -> str:
    """Merge a list of GeoTIFF files into a single JPEG file.

    Args:
    src_files (List[str]): A list of file paths to be merged.
    dest_path (str): The path to the output JPEG file.

    Returns:
    str: The path to the output JPEG file.
    """
    # Check if path to source exists
    for file in src_files:
        if not os.path.exists(file):
            raise FileNotFoundError(f"{file} not found.")
    try:
        ## create vrt(virtual world format) file
        # Create VRT file
        vrt_options = gdal.BuildVRTOptions(
            resampleAlg="mode", srcNodata=0, VRTNodata=0
        )
        print(f"dest_path: {dest_path}")
        parent_dir = os.path.dirname(dest_path)
        tmp_path = os.path.join(parent_dir,'tmp.vrt')
        print(f"tmp_path: {tmp_path}")
        
        build_vrt(tmp_path, src_files, 'max')
        
        gdal.SetConfigOption('GDAL_VRT_ENABLE_PYTHON', 'YES')

        # parent_dir = os.path.dirname(dest_path)
        # dst_path = os.path.join(parent_dir,'final.vrt')
        print(f"dest_path: {dest_path}")
        virtual_dataset = gdal.Translate(destName=dest_path, srcDS=tmp_path)
        virtual_dataset.FlushCache()
        virtual_dataset = None
        print(os.path.exists(dest_path))

        # if os.path.isfile(constants.TEMP_VRT_FILE):
        #     os.remove(constants.TEMP_VRT_FILE)
        # creates a virtual world file using all the tifs and overwrites any pre-existing .vrt
        # virtual_dataset = gdal.BuildVRT(dest_path, src_files, options=vrt_options)
        # # flushing the cache causes the vrt file to be created
        # virtual_dataset.FlushCache()
        # # reset the dataset object
        # virtual_dataset = None

#         print(f"dest_path: {dest_path}")
        # virtual_dataset = build_vrt(dest_path, src_files, 'max')
        # virtual_dataset.FlushCache()
        # virtual_dataset = None

        print(f"after new  build_vrt dest_path: {dest_path}")

        # create geotiff (.tiff) from merged vrt file
        tif_path = dest_path.replace(".vrt", ".tif")
        print(f"tif_path: {tif_path}")
        virtual_dataset = gdal.Translate(
            tif_path,
            creationOptions=["COMPRESS=LZW", "TILED=YES"],
            srcDS=dest_path,
        )
        virtual_dataset.FlushCache()
        virtual_dataset = None
        print(f"dest_path(tif_path): {tif_path}")

        if create_jpg:
            # convert .vrt to .jpg file
            virtual_dataset = gdal.Translate(
                dest_path.replace(".vrt", ".jpg"),
                creationOptions=["WORLDFILE=YES", "QUALITY=100"],
                srcDS=dest_path.replace(".vrt", ".tif"),
            )
            virtual_dataset.FlushCache()
            virtual_dataset = None
            print(f"dest_path: {dest_path.replace('.vrt', '.jpg')}")

        return dest_path
    except Exception as e:
        print(e)
        raise e

In [ ]:
# create geotiff (.tiff) from merged vrt file
# dest_path = r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010\merged_multispectral_fruit.vrt'
# tif_path = dest_path.replace(".vrt", ".tif")
# virtual_dataset = gdal.Translate(
#     tif_path,
#     creationOptions=["COMPRESS=LZW", "TILED=YES"],
#     srcDS=dest_path,
# )
# virtual_dataset.FlushCache()
# virtual_dataset = None

In [ ]:
from glob import glob

def merge_files(src_files: str, dest_path: str, create_jpg: bool = True) -> str:
    """Merge a list of GeoTIFF files into a single JPEG file.

    Args:
    src_files (List[str]): A list of file paths to be merged.
    dest_path (str): The path to the output JPEG file.

    Returns:
    str: The path to the output JPEG file.
    """
    # Check if path to source exists
    for file in src_files:
        if not os.path.exists(file):
            raise FileNotFoundError(f"{file} not found.")
    try:
        ## create vrt(virtual world format) file
        # Create VRT file
        vrt_options = gdal.BuildVRTOptions(
            resampleAlg="mode", srcNodata=0, VRTNodata=0
        )
        print(f"dest_path: {dest_path}")
        parent_dir = os.path.dirname(dest_path)
        tmp_path = os.path.join(parent_dir,'tmp.vrt')
        print(f"tmp_path: {tmp_path}")
        
        build_vrt(tmp_path, src_files, 'max')
        
        gdal.SetConfigOption('GDAL_VRT_ENABLE_PYTHON', 'YES')

        print(f"dest_path: {dest_path}")
        virtual_dataset = gdal.Translate(destName=dest_path, srcDS=tmp_path)
        virtual_dataset.FlushCache()
        virtual_dataset = None
        print(os.path.exists(dest_path))


        print(f"after new  build_vrt dest_path: {dest_path}")

        # create geotiff (.tiff) from merged vrt file
        tif_path = dest_path.replace(".vrt", ".tif")
        print(f"tif_path: {tif_path}")
        virtual_dataset = gdal.Translate(
            tif_path,
            creationOptions=["COMPRESS=LZW", "TILED=YES"],
            srcDS=dest_path,
        )
        virtual_dataset.FlushCache()
        virtual_dataset = None
        print(f"dest_path(tif_path): {tif_path}")

        if create_jpg:
            # convert .vrt to .jpg file
            virtual_dataset = gdal.Translate(
                dest_path.replace(".vrt", ".jpg"),
                creationOptions=["WORLDFILE=YES", "QUALITY=100"],
                srcDS=dest_path.replace(".vrt", ".tif"),
            )
            virtual_dataset.FlushCache()
            virtual_dataset = None
            print(f"dest_path: {dest_path.replace('.vrt', '.jpg')}")

        return dest_path
    except Exception as e:
        print(e)
        raise e

src_path=r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010'
tif_files = glob(os.path.join(src_path, "*.tif"))
tif_files = [file for file in tif_files if "merged_multispectral" not in file]

dst_path = os.path.join(src_path, "merged_multispectral_fruit_2.vrt")
merged_file = merge_files(tif_files, dst_path, create_jpg=True)

In [ ]:
os.path.dirname(dst_path)

In [ ]:
tif_files

In [ ]:
dst_path

In [ ]:
from src.seg2map import common
from glob import glob
import os
session_year_path =  r'C:\1_USGS\4_seg2map\seg2map\sessions\new_official_test_site_aaai-buildings\2010'
png_files = glob(os.path.join(session_year_path, "*.png"))
png_files = common.filter_files(png_files,[".*overlay.*"])

In [ ]:
png_files

In [ ]:
from PIL import Image
import numpy as np

def array_to_jpg(array, filename):
    # Ensure that the array has 3 channels (i.e. is an RGB image)
    if array.shape[2] != 3:
        raise ValueError("Array must have 3 channels (i.e. be an RGB image)")
    
    # Normalize the array values to the range [0, 255]
    array = (array * 255).astype(np.uint8)
    
    # Create a Pillow image from the array
    image = Image.fromarray(array)
    
    # Save the image as a JPEG file
    image.save(filename, "JPEG")

In [ ]:
# Example usage
array = np.random.rand(100, 100, 3)  # Generate a random RGB image
filename = "example.jpg"  # Filename to save the JPEG image to
array_to_jpg(array, filename)  # Convert the array to a JPEG image and save it

In [ ]:
from PIL import Image
import numpy as np

def isolate_colors(input_filename, output_prefix):
    # Open the input image and convert it to a NumPy array
    with Image.open(input_filename) as image:
        array = np.array(image)
    
    # Find the unique colors in the image
    unique_colors = np.unique(array.reshape(-1, array.shape[2]), axis=0)
    
    # For each unique color, create a mask that isolates that color
    for i, color in enumerate(unique_colors):
        # Create a mask that isolates pixels of this color
        mask = np.all(array == color, axis=2)
        
        # Convert the mask to a PIL image and save it
        mask_image = Image.fromarray(mask.astype(np.uint8) * 255)
        mask_image.save(f"{output_prefix}_{i}.jpg", "JPEG")

In [ ]:
from PIL import Image
import numpy as np

def isolate_colors(input_filename, output_prefix):
    # Open the input image and convert it to a NumPy array
    with Image.open(input_filename) as image:
        array = np.array(image)
    
    # Find the unique colors in the image
    unique_colors = np.unique(array.reshape(-1, array.shape[2]), axis=0)
    
    # Create a new array for the output image, initialized to transparent black
    output_array = np.zeros_like(array, dtype=np.uint8)
    output_array[..., 3] = 0
    
    # For each unique color, create a mask that isolates that color
    for i, color in enumerate(unique_colors):
        # Create a mask that isolates pixels of this color
        mask = np.all(array == color, axis=2)
        
        # Apply the mask to the output array, setting pixels of this color to opaque
        output_array[mask, :] = color
        output_array[mask, 3] = 255
    
    # Convert the output array to a PIL image and save it
    output_image = Image.fromarray(output_array)
    output_image.save(f"{output_prefix}.jpg", "JPEG")

In [ ]:
from PIL import Image
import numpy as np

def isolate_colors(input_filename, output_prefix):
    # Open the input image and convert it to a NumPy array
    with Image.open(input_filename) as image:
        array = np.array(image)
    
    # Find the unique colors in the image
    unique_colors = np.unique(array.reshape(-1, array.shape[2]), axis=0)
    
    # For each unique color, create a mask that isolates that color
    for i, color in enumerate(unique_colors):
        # Create a mask that isolates pixels of this color
        mask = np.all(array == color, axis=2)
        
        # Create a new output image with pixels of this color set to the original color and other pixels set to black
        output_array = np.zeros_like(array, dtype=np.uint8)
        output_array[mask] = color
        output_image = Image.fromarray(output_array)
        
        # Save the output image
        output_image.save(f"{output_prefix}_{i}.jpg", "JPEG")

In [ ]:
# Example usage
input_filename = r'C:\1_USGS\4_seg2map\seg2map\sessions\new_official_test_site\2010\multiband0_USDA_NAIP_DOQQ_m_4012415_ne_10_1_20100629.png'
output_prefix = "output_test_two"  # Prefix to use for the output filenames
isolate_colors(input_filename, output_prefix)  # Isolate each color in the input image and save the masks

In [ ]:
'output_test_11.jpg'

In [ ]:

from PIL import Image
import numpy as np

# Example usage
input_filename =r'C:\1_USGS\4_seg2map\seg2map\sessions\test_2_\2010\multiband0_USDA_NAIP_DOQQ_m_4012415_ne_10_1_20100629.png'
# input_filename = r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010\merged_multispectral_fruit.jpg'
output_prefix = "output_test_two"  # Prefix to use for the output filenames
# isolate_colors(input_filename, output_prefix)  # Isolate each color in the input image and save the masks

# Load the input image
with Image.open(input_filename) as image:
    # Convert the image to a NumPy array
    array3 = np.array(image)

In [ ]:
from PIL import Image
import numpy as np

# Example usage
input_filename =r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010\multiband0_USDA_NAIP_DOQQ_m_4012415_ne_10_1_20100629.png'

# Load the input image
with Image.open(input_filename) as image:
    # Convert the image to a NumPy array
    array3 = np.array(image)
reshaped_array3 =array3.reshape(-1, array3.shape[2])
unqiue_colors = np.unique(reshaped_array3,axis=0)
print(unqiue_colors)

In [ ]:
reshaped_array3 =array3.reshape(-1, array3.shape[2])
reshaped_array3

In [ ]:
unqiue_colors = np.unique(reshaped_array3,axis=0)
unqiue_colors

In [ ]:
reshaped_array3 =array3.reshape(-1, array3.shape[2])
reshaped_array3

In [ ]:
reshaped_array3.size/3

In [ ]:
unqiue_colors = np.unique(reshaped_array3,axis=0)
unqiue_colors

In [ ]:
len(unqiue_colors)

In [ ]:
from PIL import Image
import numpy as np
input_filename = r'C:\1_USGS\4_seg2map\seg2map\sessions\new_official_test_site\2010\multiband1_USDA_NAIP_DOQQ_m_4012415_ne_10_1_20100629.png'
# Load the input image
with Image.open('multiband2_USDA_NAIP_DOQQ_m_4012415_ne_10_1_20100629.png') as image:
    # Convert the image to a NumPy array
    array2 = np.array(image)

In [ ]:
reshaped_array2 =array2.reshape(-1, array2.shape[2])
reshaped_array2

In [ ]:
reshaped_array2[1000:1010]

In [ ]:
np.unique(reshaped_array2,axis=0)

In [ ]:
from PIL import Image
import numpy as np

path = r'C:\1_USGS\4_seg2map\seg2map\sessions\mode_experiment\2010\merged_multispectral_fruit.jpg'
# Load the input image
with Image.open(path) as image:
    # Convert the image to a NumPy array
    array = np.array(image)

In [ ]:
arr[0][0]

In [ ]:
array[0][0:10]

In [ ]:
reshaped_array =array.reshape(-1, array.shape[2])
reshaped_array

In [ ]:
reshaped_array[0]

In [ ]:
np.unique(reshaped_array[0:10],axis=0)

In [ ]:
# np.unique(reshaped_array[0:5000],axis=0)

In [ ]:
np.unique(reshaped_array[0:5000])

In [ ]:
reshaped_array.shape

In [ ]:
reshaped_array[0:10]

In [ ]:
np.max(array[0][0])

In [ ]:
np.max(array[0][0])

In [ ]:
array.shape

In [ ]:
array[0][0]